<h2>1) Listado Persona Jurídica → DataFrame con URL Ficha</h2>

In [17]:
import re, time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Listado PJ
LIST_URL_JUR = "https://www.cmfchile.cl/institucional/mercados/consulta.php?mercado=S&consulta=CSJUR"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8",
}

# Descargar
resp = requests.get(LIST_URL_JUR, headers=HEADERS, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

# Ubicar la tabla con encabezado R.U.T. | Entidad | Vigencia
def headers_de_tabla(tbl):
    thead = tbl.find("thead")
    if thead:
        hdrs = [th.get_text(" ", strip=True) for th in thead.find_all("th")]
    else:
        first_tr = tbl.find("tr")
        hdrs = [td.get_text(" ", strip=True) for td in first_tr.find_all(["th","td"])] if first_tr else []
    return hdrs

target = None
for tbl in soup.find_all("table"):
    hdrs = headers_de_tabla(tbl)
    norm = [re.sub(r"\s+", " ", h.strip()) for h in hdrs]
    if len(norm) >= 3 and norm[0].upper().startswith("R.U.T") and norm[1].lower()=="entidad":
        target = tbl
        break
if not target:
    raise RuntimeError("No encontré la tabla del listado PJ.")

# Extraer filas
rows = []
tbody = target.find("tbody")
trs = tbody.find_all("tr") if tbody else target.find_all("tr")[1:]
for tr in trs:
    tds = tr.find_all("td")
    if len(tds) < 3:
        continue
    rut = tds[0].get_text(" ", strip=True)
    entidad_td = tds[1]
    entidad = entidad_td.get_text(" ", strip=True)
    a = entidad_td.find("a")
    href = a.get("href") if a and a.get("href") else ""
    # Resolver relativo contra el listado (así conserva /institucional/mercados/)
    url_ficha = urljoin(LIST_URL_JUR, href) if href else ""
    vig = tds[2].get_text(" ", strip=True)
    rows.append({"R.U.T.": rut, "Entidad": entidad, "Vigencia": vig, "URL Ficha": url_ficha})

df = pd.DataFrame(rows)
pd.set_option("display.max_colwidth", None)
display(df.head(10))
print("Total en listado PJ:", len(df))


,R.U.T.,Entidad,Vigencia,URL Ficha
0,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
1,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
2,77672889-6,SICURO CORREDORES DE SEGUROS SPA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=77672889&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOgAAf&control=svs&pestania=1
3,76815712-K,123 CORREDORES DE SEGUROS SPA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76815712&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWO/AAJ&control=svs&pestania=1
4,78082726-2,125 GLOBAL INSURANCE SpA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=78082726&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWPnAAI&control=svs&pestania=1
5,78159157-2,4X4 CORREDORES DE SEGURO SpA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=78159157&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWPcAAK&control=svs&pestania=1
6,76026159-9,A & A MAS LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76026159&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAAHaAAR&control=svs&pestania=1
7,76204605-9,A & B CORREDORES DE SEGUROS SPA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76204605&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAAHgAAP&control=svs&pestania=1
8,76153122-0,A & H PRODUCTORA DE SEGUROS LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76153122&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAAHrAAL&control=svs&pestania=1
9,76406444-5,A & S CORREDORES DE SEGUROS ASOCIADOS LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76406444&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAAH9AAI&control=svs&pestania=1


Total en listado PJ: 1967


<h2>2) Normalizar URL de ficha (forzar ruta correcta y conservar row=) + helpers</h2>

In [21]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

def resolver_url_ficha(u: str, list_url: str = LIST_URL_JUR) -> str:
    # urljoin ya lo hicimos; aquí solo forzamos el path correcto y preservamos parámetros
    from urllib.parse import urljoin
    absu = urljoin(list_url, u)
    p = urlparse(absu)
    path_ok = "/institucional/mercados/entidad.php"
    if p.path.endswith("/entidad.php") and p.path != path_ok:
        p = p._replace(path=path_ok)
    qs = parse_qs(p.query, keep_blank_values=True)
    new_q = urlencode(qs, doseq=True)
    return urlunparse((p.scheme, p.netloc, p.path, p.params, new_q, p.fragment))

def set_pestania(url: str, pestaña: int = 1) -> str:
    p = urlparse(url)
    qs = parse_qs(p.query, keep_blank_values=True)
    qs["pestania"] = [str(pestaña)]
    return urlunparse((p.scheme, p.netloc, p.path, p.params, urlencode(qs, doseq=True), p.fragment))

def get_soup(url, tries=3, pause=0.7):
    last = None
    for i in range(tries):
        try:
            r = requests.get(url, headers=HEADERS, timeout=30)
            r.raise_for_status()
            return BeautifulSoup(r.text, "html.parser")
        except Exception as e:
            last = e
            time.sleep(pause*(i+1))
    raise last

# Normalizar columna URL Ficha
df["URL Ficha"] = df["URL Ficha"].apply(lambda u: resolver_url_ficha(u) if isinstance(u, str) else "")
display(df.head(3)[["URL Ficha"]])


,URL Ficha
0,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
1,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
2,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=77672889&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOgAAf&control=svs&pestania=1


<h2>3) Parser “canónico” de Identificación (siempre mismas columnas)</h2>

In [24]:
import numpy as np

CANON_COLS = [
    "rut","nombre_razon_social","nombre_fantasia","vigencia","tipo_persona",
    "numero_inscripcion","tipo_doc_nombramiento","nro_doc_nombramiento",
    "fecha_doc_nombramiento","telefono","domicilio","comuna","ciudad","region","casilla"
]

KEYMAP = [
    (re.compile(r"^r\.?u\.?t\.?$", re.I), "rut"),
    (re.compile(r"raz[oó]n\s*social|^nombre\s*/?\s*raz[oó]n", re.I), "nombre_razon_social"),
    (re.compile(r"nombre\s*de\s*fantas[ií]a", re.I), "nombre_fantasia"),
    (re.compile(r"^vigencia$", re.I), "vigencia"),
    (re.compile(r"tipo\s*de\s*persona", re.I), "tipo_persona"),
    (re.compile(r"(n[uú]mero|nº|n°)\s*de\s*inscripci[oó]n", re.I), "numero_inscripcion"),
    (re.compile(r"tipo\s*documento\s*nombramiento", re.I), "tipo_doc_nombramiento"),
    (re.compile(r"(n[uú]m\.?|nro\.?|n°)\s*documento\s*nombramiento", re.I), "nro_doc_nombramiento"),
    (re.compile(r"fecha\s*documento\s*nombramiento", re.I), "fecha_doc_nombramiento"),
    (re.compile(r"^tel[eé]fono$", re.I), "telefono"),
    (re.compile(r"^domicilio$", re.I), "domicilio"),
    (re.compile(r"^comuna$", re.I), "comuna"),
    (re.compile(r"^ciudad$", re.I), "ciudad"),
    (re.compile(r"^regi[oó]n$", re.I), "region"),
    (re.compile(r"^casilla$", re.I), "casilla"),
]

def _clean_text(x: str) -> str:
    x = "" if x is None else str(x)
    return re.sub(r"\s+", " ", x).strip()

def parse_identificacion_tidy(soup) -> dict:
    # localizar la sección
    title = None
    for tag in soup.find_all(["h1","h2","h3","h4","h5"]):
        if "identificacion" in tag.get_text(" ", strip=True).lower():
            title = tag
            break
    table = title.find_next("table") if title else None
    if not table:
        for tbl in soup.find_all("table"):
            if "R.U.T" in tbl.get_text(" ", strip=True):
                table = tbl
                break
    if not table:
        return {c: "" for c in CANON_COLS}

    # pares clave/valor
    kv = {}
    for tr in table.find_all("tr"):
        tds = tr.find_all(["td","th"])
        if len(tds) >= 2:
            k = _clean_text(tds[0].get_text(" ", strip=True))
            v = _clean_text(tds[1].get_text(" ", strip=True))
            if k:
                kv[k] = v

    # mapear a claves canónicas
    out = {c: "" for c in CANON_COLS}
    for raw_key, val in kv.items():
        for rx, canon in KEYMAP:
            if rx.search(raw_key):
                out[canon] = val
                break
    return out


<h2>4) Probar con 5 fichas</h2>

In [27]:
muestras = df.head(5).copy()
detalles = []
for _, r in muestras.iterrows():
    url_ident = set_pestania(r["URL Ficha"], pestaña=1)
    print("Probando:", url_ident)
    s = get_soup(url_ident)
    d = parse_identificacion_tidy(s)
    d["url_ficha"] = url_ident
    detalles.append(d)
    time.sleep(0.5)

detalles_df = pd.DataFrame(detalles)
pd.set_option("display.max_colwidth", None)
display(detalles_df)


Probando: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
Probando: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
Probando: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=77672889&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOgAAf&control=svs&pestania=1
Probando: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76815712&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWO%2FAAJ&control=svs&pestania=1
Probando: https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=78082726&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWPnAAI&control=svs&pestania=1


,rut,nombre_razon_social,nombre_fantasia,vigencia,tipo_persona,numero_inscripcion,tipo_doc_nombramiento,nro_doc_nombramiento,fecha_doc_nombramiento,telefono,domicilio,comuna,ciudad,region,casilla,url_ficha
0,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,,Corredor No Previs. Con Registro Vigente,Persona Jurídica,7695,Certificado,,13/01/2015,,"SANTA ROSA 441, D.303, ED. CORDILLERA 1",,LOS ANDES,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
1,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,,Corredor No Previs. Con Registro Vigente,Persona Jurídica,7695,Certificado,,13/01/2015,,"SANTA ROSA 441, D.303, ED. CORDILLERA 1",,LOS ANDES,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
2,77672889-6,SICURO CORREDORES DE SEGUROS SPA,,Corredor No Previs. Con Registro Vigente,Persona Jurídica,9381,Certificado,,21/04/2023,,EXEQUIEL FERNANDEZ 620 DEPARTAMENTO 45,,SANTIAGO,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=77672889&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOgAAf&control=svs&pestania=1
3,76815712-K,123 CORREDORES DE SEGUROS SPA,,Corredor No Previs. Con Registro Vigente,Persona Jurídica,8425,Certificado,,25/10/2018,,"SUECIA 0155, OFICINA 904",,SANTIAGO,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76815712&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWO%2FAAJ&control=svs&pestania=1
4,78082726-2,125 GLOBAL INSURANCE SpA,125 GLOBAL INSURANCE SpA,Corredor No Previs. Con Registro Vigente,Persona Jurídica,9850,Certificado,,06/02/2025,,ntonio Bellet 193 Of/Depto/Casa Nº 1210,,SANTIAGO,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=78082726&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWPnAAI&control=svs&pestania=1


<h2>5) Correr todo y exportar</h2>

In [30]:
det_all = []
for i, r in df.iterrows():
    url_ident = set_pestania(r["URL Ficha"], pestaña=1)
    try:
        s = get_soup(url_ident)
        d = parse_identificacion_tidy(s)
        d["url_ficha"] = url_ident
        det_all.append(d)
    except Exception as e:
        det_all.append({"url_ficha": url_ident, "error": str(e)})
    if (i+1) % 50 == 0:
        print(f"Procesadas {i+1} fichas PJ...")
        # opcional checkpoint:
        # pd.DataFrame(det_all).to_parquet(f"checkpoint_pj_{i+1}.parquet", index=False)
    time.sleep(0.4)  # pausa amable

df_ident_pj = pd.DataFrame(det_all)

# Unir con listado para conservar RUT/Entidad/Vigencia de la grilla
final_pj = df.merge(df_ident_pj, left_on="URL Ficha", right_on="url_ficha", how="left")

# Limpieza rápida de fecha/teléfono (opcional)
if "fecha_doc_nombramiento" in final_pj.columns:
    final_pj["fecha_doc_nombramiento"] = pd.to_datetime(final_pj["fecha_doc_nombramiento"], dayfirst=True, errors="coerce")
if "telefono" in final_pj.columns:
    final_pj["telefono"] = final_pj["telefono"].astype(str).str.replace(r"\s+", "", regex=True)

# Exportar
final_pj.to_csv("cmf_pj_identificacion_final.csv", index=False, encoding="utf-8-sig")
print("✅ Exportado: cmf_pj_identificacion_final.csv")
display(final_pj.head(3))


Procesadas 50 fichas PJ...
Procesadas 100 fichas PJ...
Procesadas 150 fichas PJ...
Procesadas 200 fichas PJ...
Procesadas 250 fichas PJ...
Procesadas 300 fichas PJ...
Procesadas 350 fichas PJ...
Procesadas 400 fichas PJ...
Procesadas 450 fichas PJ...
Procesadas 500 fichas PJ...
Procesadas 550 fichas PJ...
Procesadas 600 fichas PJ...
Procesadas 650 fichas PJ...
Procesadas 700 fichas PJ...
Procesadas 750 fichas PJ...
Procesadas 800 fichas PJ...
Procesadas 850 fichas PJ...
Procesadas 900 fichas PJ...
Procesadas 950 fichas PJ...
Procesadas 1000 fichas PJ...
Procesadas 1050 fichas PJ...
Procesadas 1100 fichas PJ...
Procesadas 1150 fichas PJ...
Procesadas 1200 fichas PJ...
Procesadas 1250 fichas PJ...
Procesadas 1300 fichas PJ...
Procesadas 1350 fichas PJ...
Procesadas 1400 fichas PJ...
Procesadas 1450 fichas PJ...
Procesadas 1500 fichas PJ...
Procesadas 1550 fichas PJ...
Procesadas 1600 fichas PJ...
Procesadas 1650 fichas PJ...
Procesadas 1700 fichas PJ...
Procesadas 1750 fichas PJ...
Proce

,R.U.T.,Entidad,Vigencia,URL Ficha,rut,nombre_razon_social,nombre_fantasia,vigencia,tipo_persona,numero_inscripcion,tipo_doc_nombramiento,nro_doc_nombramiento,fecha_doc_nombramiento,telefono,domicilio,comuna,ciudad,region,casilla,url_ficha
0,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,,Corredor No Previs. Con Registro Vigente,Persona Jurídica,7695,Certificado,,2015-01-13,,"SANTA ROSA 441, D.303, ED. CORDILLERA 1",,LOS ANDES,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
1,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,,Corredor No Previs. Con Registro Vigente,Persona Jurídica,7695,Certificado,,2015-01-13,,"SANTA ROSA 441, D.303, ED. CORDILLERA 1",,LOS ANDES,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
2,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,VI,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1,76457592-K,MARCELA HERRERA CORREDORES DE SEGUROS Y CIA. LIMITADA,,Corredor No Previs. Con Registro Vigente,Persona Jurídica,7695,Certificado,,2015-01-13,,"SANTA ROSA 441, D.303, ED. CORDILLERA 1",,LOS ANDES,,,https://www.cmfchile.cl/institucional/mercados/entidad.php?auth=&send=&mercado=S&rut=76457592&grupo=&tipoentidad=CSJUR&vig=VI&row=AAAwU3AAWAAAWOaAAW&control=svs&pestania=1
